# Transformation Silver → Gold

## 📋 Objectif
Créer un modèle dimensionnel (star schema) optimisé pour l'analyse.

## 🌟 Modèle créé
- **4 dimensions** : Date, Time, Turbine, Operational Status
- **1 fait** : Production éolienne

## 📦 Dépendances
- **Input** : LH_Wind_Power_Silver.wind_power
- **Output** : LH_Wind_Power_Gold (5 tables)

In [1]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 3, Finished, Available, Finished)

In [3]:
silver_table_path = "abfss://96ced640-8cdf-4c77-b4a2-8807c10c6166@onelake.dfs.fabric.microsoft.com/4cf44c04-2547-4d0d-b846-3ee048540ce1/Tables/wind_power"

df = spark.read.format("delta").load(silver_table_path)

print(f"📊 Données chargées : {df.count()} lignes")

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 5, Finished, Available, Finished)

📊 Données chargées : 864 lignes


In [4]:
# Créer dim_date avec valeurs uniques
date_dim = (df
    .select("date", "day", "month", "quarter", "year")
    .distinct()
    .withColumnRenamed("date", "date_id")
)

print(f"📅 dim_date : {date_dim.count()} dates uniques")
date_dim.show(5)

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 6, Finished, Available, Finished)

📅 dim_date : 2 dates uniques
+-------------------+---+-----+-------+----+
|            date_id|day|month|quarter|year|
+-------------------+---+-----+-------+----+
|2024-06-16 00:00:00| 16|    6|      2|2024|
|2024-06-15 00:00:00| 15|    6|      2|2024|
+-------------------+---+-----+-------+----+



In [5]:
# Créer dim_time avec valeurs uniques
time_dim = (df
    .select("time", "hour_of_day", "minute_of_hour", "second_of_minute", "time_period")
    .distinct()
    .withColumnRenamed("time", "time_id")
)

print(f"⏰ dim_time : {time_dim.count()} temps uniques")
time_dim.show(5)

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 7, Finished, Available, Finished)

⏰ dim_time : 144 temps uniques
+--------+-----------+--------------+----------------+-----------+
| time_id|hour_of_day|minute_of_hour|second_of_minute|time_period|
+--------+-----------+--------------+----------------+-----------+
|10:10:00|         10|            10|               0|    Morning|
|11:10:00|         11|            10|               0|    Morning|
|18:10:00|         18|            10|               0|    Evening|
|18:30:00|         18|            30|               0|    Evening|
|19:10:00|         19|            10|               0|    Evening|
+--------+-----------+--------------+----------------+-----------+
only showing top 5 rows



In [6]:
# Créer dim_turbine avec ID auto-incrémenté
turbine_dim = (df
    .select("turbine_name", "capacity", "location_name", "latitude", "longitude", "region")
    .distinct()
    .withColumn("turbine_id", 
        row_number().over(
            Window.orderBy("turbine_name", "capacity", "location_name")
        )
    )
)

print(f"🌬️ dim_turbine : {turbine_dim.count()} turbines")
turbine_dim.show()

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 8, Finished, Available, Finished)

🌬️ dim_turbine : 3 turbines
+------------+--------+-------------+--------+---------+--------+----------+
|turbine_name|capacity|location_name|latitude|longitude|  region|turbine_id|
+------------+--------+-------------+--------+---------+--------+----------+
|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|         1|
|   Turbine B|    2000|   Location 2| 36.7783|-119.4179|Region B|         2|
|   Turbine C|    2500|   Location 3| 40.7128|  -74.006|Region C|         3|
+------------+--------+-------------+--------+---------+--------+----------+



In [7]:
# Créer dim_operational_status avec ID
operational_status_dim = (df
    .select("status", "responsible_department")
    .distinct()
    .withColumn("status_id", 
        row_number().over(
            Window.orderBy("status", "responsible_department")
        )
    )
)

print(f"📊 dim_operational_status : {operational_status_dim.count()} statuts")
operational_status_dim.show()

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 9, Finished, Available, Finished)

📊 dim_operational_status : 6 statuts
+--------------------+----------------------+---------+
|              status|responsible_department|status_id|
+--------------------+----------------------+---------+
|               Fault|           Engineering|        1|
|          Inspection|           Maintenance|        2|
|             Offline|            Operations|        3|
|              Online|            Operations|        4|
|Preventive Mainte...|           Maintenance|        5|
|Reactive Maintenance|           Maintenance|        6|
+--------------------+----------------------+---------+



In [10]:
# Joindre pour récupérer les clés étrangères
df_with_keys = (df
    .join(turbine_dim, 
          ["turbine_name", "capacity", "location_name", "latitude", "longitude", "region"], 
          "left")
    .join(operational_status_dim, 
          ["status", "responsible_department"], 
          "left")
)
df_with_keys.show(5)

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 12, Finished, Available, Finished)

+------+----------------------+------------+--------+-------------+--------+---------+--------+-------------+-------------------+--------------+----------+---------------+---+-----+-------+----+--------+-----------+--------------+----------------+-----------+----------+---------+
|status|responsible_department|turbine_name|capacity|location_name|latitude|longitude|  region|production_id|               date|wind_direction|wind_speed|energy_produced|day|month|quarter|year|    time|hour_of_day|minute_of_hour|second_of_minute|time_period|turbine_id|status_id|
+------+----------------------+------------+--------+-------------+--------+---------+--------+-------------+-------------------+--------------+----------+---------------+---+-----+-------+----+--------+-----------+--------------+----------------+-----------+----------+---------+
|Online|            Operations|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|         6697|2024-06-16 00:00:00|            NW|     24.19|  

In [11]:
# Table de faits avec uniquement clés et mesures
fact_table = (df_with_keys
    .select(
        "production_id",
        col("date").alias("date_id"),
        col("time").alias("time_id"),
        "turbine_id",
        "status_id",
        "wind_speed",
        "wind_direction",
        "energy_produced"
    )
)

print(f"📈 fact_wind_power : {fact_table.count()} mesures")
fact_table.show(10)

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 13, Finished, Available, Finished)

📈 fact_wind_power : 864 mesures
+-------------+-------------------+--------+----------+---------+----------+--------------+---------------+
|production_id|            date_id| time_id|turbine_id|status_id|wind_speed|wind_direction|energy_produced|
+-------------+-------------------+--------+----------+---------+----------+--------------+---------------+
|         6697|2024-06-16 00:00:00|12:00:00|         1|        4|     24.19|            NW|         598.44|
|         6698|2024-06-16 00:00:00|12:00:00|         2|        4|      9.72|             S|         906.98|
|         6699|2024-06-16 00:00:00|12:00:00|         3|        4|      4.71|            NW|        1541.91|
|         6700|2024-06-16 00:00:00|12:10:00|         1|        4|       3.0|             N|        1794.64|
|         6701|2024-06-16 00:00:00|12:10:00|         2|        4|     15.55|             W|        1802.04|
|         6702|2024-06-16 00:00:00|12:10:00|         3|        4|      3.96|            SW|         955.

In [12]:
base_path = "abfss://96ced640-8cdf-4c77-b4a2-8807c10c6166@onelake.dfs.fabric.microsoft.com/7385b96c-af83-4c53-94ae-4b6b59a01418/Tables"

gold_date_dim_path = f"{base_path}/dim_date"
gold_time_dim_path = f"{base_path}/dim_time"
gold_turbine_dim_path = f"{base_path}/dim_turbine"
gold_operational_status_dim_path = f"{base_path}/dim_operational_status"
gold_fact_table_path = f"{base_path}/fact_wind_power"

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 14, Finished, Available, Finished)

In [13]:
# Sauvegarder toutes les tables
date_dim.write.format("delta").mode("overwrite").save(gold_date_dim_path)
time_dim.write.format("delta").mode("overwrite").save(gold_time_dim_path)
turbine_dim.write.format("delta").mode("overwrite").save(gold_turbine_dim_path)
operational_status_dim.write.format("delta").mode("overwrite").save(gold_operational_status_dim_path)
fact_table.write.format("delta").mode("overwrite").save(gold_fact_table_path)

print("✅ Toutes les tables créées dans Gold !")

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 15, Finished, Available, Finished)

✅ Toutes les tables créées dans Gold !


In [15]:
from pyspark.sql.functions import count, when

print("=== 🔍 Vérification intégrité référentielle ===\n")

# Vérifier les dates
fact_dates = fact_table.select("date_id").distinct().count()
dim_dates = date_dim.count()
print(f"📅 Dates - Fait: {fact_dates}, Dim: {dim_dates}")

# Vérifier les turbines
fact_turbines = fact_table.select("turbine_id").distinct().count()
dim_turbines = turbine_dim.count()
print(f"🌬️ Turbines - Fait: {fact_turbines}, Dim: {dim_turbines}")

# Vérifier les clés nulles
null_check = fact_table.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["date_id", "time_id", "turbine_id", "status_id"]
])
print("\n❌ Clés nulles dans fact :")
null_check.show()

StatementMeta(, 5ad9038b-c957-4b30-85a7-275e7666a7b9, 17, Finished, Available, Finished)

=== 🔍 Vérification intégrité référentielle ===

📅 Dates - Fait: 2, Dim: 2
🌬️ Turbines - Fait: 3, Dim: 3

❌ Clés nulles dans fact :
+-------+-------+----------+---------+
|date_id|time_id|turbine_id|status_id|
+-------+-------+----------+---------+
|      0|      0|         0|        0|
+-------+-------+----------+---------+

